# Probability mass functions


In [66]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkplot.py")

## `Pmf`s

Another way to represent a distribution is a **probability mass
function** (PMF), which maps from each value to its probability. A
**probability** is a frequency expressed as a fraction of the sample
size, `n`. To get from frequencies to probabilities, we divide through
by `n`, which is called **normalization**.

Given a Hist, we can make a dictionary that maps from each value to its
probability:

In [67]:
from thinkstats2 import Hist

hist = Hist([1, 2, 2, 3, 5])
hist

In [68]:
n = hist.total()
d = {}
for x, freq in hist.items():
    d[x] = freq / n

d

Or we can use the `Pmf` class provided by `thinkstats2`. Like Hist, the
`Pmf` constructor can take a list, pandas `Series`, dictionary, Hist, or
another `Pmf` object. Here's an example with a simple list:

In [94]:
from empiricaldist import Pmf

pmf = Pmf.from_seq([1, 2, 2, 3, 5])
pmf

The `Pmf` is normalized so total probability is 1.

`Pmf` and `Hist` objects are similar in many ways; in fact, they inherit
many of their methods from a common parent class. For example, the
methods `values` and `items` work the same way for both. The biggest
difference is that a Hist maps from values to integer counters; a `Pmf`
maps from values to floating-point probabilities.

To look up the probability associated with a value, use parentheses to call the `Pmf` like a function.

In [95]:
pmf(2)

To assign a probability to a quantity, you have to use the bracket operator.

In [96]:
pmf[2] = 0.2
pmf(2)

You can modify an existing `Pmf` by incrementing the probability
associated with a value:

In [97]:
pmf[2] += 0.3
pmf(2)

Or you can multiply a probability by a factor:

In [98]:
pmf[2] *= 0.5
pmf(2)

If you modify a `Pmf`, the result may not be normalized; that is, the
probabilities may no longer add up to 1. To check, you can call `sum`,
which returns the sum of the probabilities:

In [101]:
pmf.sum()

To renormalize, call `normalize`:

In [102]:
pmf.normalize()
pmf.sum()

`Pmf` objects provide a `copy` method so you can make and modify a copy
without affecting the original.

In [103]:
pmf.copy()

A note on notation: I use `Pmf` for the name of the class, `pmf` for an instance of
the class, and PMF for the mathematical concept of a probability mass
function.

## Plotting PMFs

`thinkplot` provides two ways to plot a `pmf`:

-   To plot a `Pmf` as a bar graph, you can use `thinkplot.hist`. Bar
    graphs are most useful if the number of values in the `Pmf` is small.

-   To plot a `Pmf` as a step function, you can use `thinkplot.pmf`. This
    option is most useful if there are a large number of values and the
    `Pmf` is smooth. This function also works with `Hist` objects.

The following figure shows PMFs of pregnancy length for first babies and others using bar graphs.

In [104]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/2002FemPreg.dct")
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/2002FemPreg.dat.gz")

In [105]:
import nsfg

live, firsts, others = nsfg.make_frames()

In [107]:
first_pmf = Pmf.from_seq(firsts.prglngth, name="firsts")
other_pmf = Pmf.from_seq(others.prglngth, name="others")

In [121]:
def pmf_bar_plots(pmf1, pmf2, width=0.45, **options):
    pmf1.bar(align="edge", width=-width, **options)
    pmf2.bar(align="edge", width=width, **options)

In [129]:
import thinkplot

width = 0.45
pmf_bar_plots(first_pmf, other_pmf)
thinkplot.config(xlabel="weeks", ylabel="probability", xlim=[24, 46])

The following figure shows the same PMFs as step functions.

In [130]:
first_pmf.plot()
other_pmf.plot()
thinkplot.config(xlabel="weeks", ylabel="probability", xlim=[24, 46])

By plotting the PMF instead of the histogram, we can compare the two
distributions without being mislead by the difference in sample size.
Based on this figure, first babies seem to be less likely than others to
arrive on time (week 39) and more likely to be late (weeks 41 and 42).

## Other visualizations

Histograms and PMFs are useful while you are exploring data and trying
to identify patterns and relationships. Once you have an idea what is
going on, a good next step is to design a visualization that makes the
patterns you have identified as clear as possible.

In the NSFG data, the biggest differences in the distributions are near
the mode. So it makes sense to zoom in on that part of the graph, and to
transform the data to emphasize differences:

In [112]:
weeks = range(35, 46)
diffs = []
for week in weeks:
    p1 = first_pmf(week)
    p2 = other_pmf(week)
    diff = 100 * (p1 - p2)
    diffs.append(diff)
thinkplot.bar(weeks, diffs)
thinkplot.config(xlabel="Weeks", ylabel="Difference (percentage points)")

In this code, `weeks` is the range of weeks; `diffs` is the difference
between the two PMFs in percentage points.
The following figure shows the result as a bar chart.
This figure makes the pattern clearer: first babies are less likely to
be born in week 39, and somewhat more likely to be born in weeks 41 and
42.

For now we should hold this conclusion only tentatively. We used the
same dataset to identify an apparent difference and then chose a
visualization that makes the difference apparent. We can't be sure this
effect is real; it might be due to random variation. We'll address this
concern later.

## The class size paradox

Before we go on, I want to demonstrate one kind of computation you can
do with `Pmf` objects; I call this example the "class size paradox."

At many American colleges and universities, the student-to-faculty ratio
is about 10:1. But students are often surprised to discover that their
average class size is bigger than 10. There are two reasons for the
discrepancy:

-   Students typically take 4--5 classes per semester, but professors
    often teach 1 or 2.

-   The number of students who enjoy a small class is small, but the
    number of students in a large class is (ahem!) large.

The first effect is obvious, at least once it is pointed out; the second
is more subtle. Let's look at an example. Suppose that a college offers
65 classes in a given semester, with the following distribution of
sizes:

If you ask the Dean for the average class size, he would construct a
PMF, compute the mean, and report that the average class size is 23.7.
Here's the code:

In [113]:
d = {7: 8, 12: 8, 17: 14, 22: 4, 27: 6, 32: 12, 37: 8, 42: 3, 47: 2}

pmf = Pmf.from_seq(d, name="actual")
print("mean", pmf.mean())

But if you survey a group of students, ask them how many students are in
their classes, and compute the mean, you would think the average class
was bigger. Let's see how much bigger.

First, I compute the distribution as observed by students, where the
probability associated with each class size is "biased" by the number of
students in the class.

In [117]:
def bias_pmf(pmf, name):
    new_pmf = pmf.copy()
    new_pmf.name = name

    for x, p in pmf.items():
        new_pmf[x] *= x
    new_pmf.normalize()
    return new_pmf

For each class size, `x`, we multiply the probability by `x`, the number
of students who observe that class size. The result is a new `Pmf` that
represents the biased distribution.

Now we can plot the actual and observed distributions:

In [118]:
biased_pmf = bias_pmf(pmf, name="observed")

In [131]:
pmf_bar_plots(pmf, biased_pmf)
thinkplot.config(xlabel="class size", ylabel="PMF")

The following figure shows the result. In the biased distribution
there are fewer small classes and more large ones. The mean of the
biased distribution is 29.1, almost 25% higher than the actual mean.

It is also possible to invert this operation. Suppose you want to find
the distribution of class sizes at a college, but you can't get reliable
data from the Dean. An alternative is to choose a random sample of
students and ask how many students are in their classes.

The result would be biased for the reasons we've just seen, but you can
use it to estimate the actual distribution. Here's the function that
unbiases a `Pmf`:

In [132]:
def unbias_pmf(pmf, name):
    new_pmf = pmf.copy()
    new_pmf.name = name

    for x, p in pmf.items():
        new_pmf[x] /= x

    new_pmf.normalize()
    return new_pmf

It's similar to `unbias_pmf`; the only difference is that it divides each
probability by `x` instead of multiplying.

## `DataFrame` indexing

In Chapter 2 we read a Pandas `DataFrame` and used it to select
and modify data columns. Now let's look at row selection. To start, I
create a NumPy array of random numbers and use it to initialize a
`DataFrame`:

In [134]:
import numpy as np
import pandas as pd

array = np.random.random((4, 2))
df = pd.DataFrame(array)
df

By default, the rows and columns are numbered starting at zero, but you
can provide column names:

In [135]:
columns = ["A", "B"]
df = pd.DataFrame(array, columns=columns)
df

You can also provide row names. The set of row names is called the
**index**; the row names themselves are called **labels**.

In [136]:
index = ["a", "b", "c", "d"]
df = pd.DataFrame(array, columns=columns, index=index)
df

As we saw in the previous chapter, simple indexing selects a column,
returning a `Series`:

In [137]:
df["A"]

To select a row by label, you can use the `loc` attribute, which returns
a `Series`:

In [138]:
df.loc["a"]

If you know the integer position of a row, rather than its label, you
can use the `iloc` attribute, which also returns a `Series`.

In [140]:
df.iloc[0]

`loc` can also take a list of labels; in that case, the result is a
`DataFrame`.

In [141]:
indices = ["a", "c"]
df.loc[indices]

Finally, you can use a slice to select a range of rows by label:

In [142]:
df["a":"c"]

Or by integer position:

In [143]:
df[0:2]

The result in either case is a `DataFrame`, but notice that the first
result includes the end of the slice; the second doesn't.

My advice: if your rows have labels that are not simple integers, use
the labels consistently and avoid using integer positions.

## Glossary

-   **Probability mass function (PMF)**: a representation of a
    distribution as a function that maps from values to probabilities.

-   **probability**: A frequency expressed as a fraction of the sample
    size.

-   **normalization**: The process of dividing a frequency by a sample
    size to get a probability.

-   **index**: In a Pandas `DataFrame`, the index is a special column that
    contains the row labels.

## Exercises

**Exercise:** In Section XXX we computed the mean of a sample by adding up
the elements and dividing by n.  If you are given a PMF, you can
still compute the mean, but the process is slightly different:

$$ \bar x = \sum_i p_i~x_i $$

where the $x_i$ are the unique values in the PMF and $p_i=PMF(x_i)$.
Similarly, you can compute variance like this:

$$ S^2 = \sum_i p_i~(x_i - \bar x)^2 $$
 
Write functions called `pmf_mean` and `pmf_var` that take a
`Pmf` object and compute the mean and variance.  To test these methods,
check that they are consistent with the methods `mean` and `var`
provided by `Pmf`.

In [150]:
pmf = Pmf.from_seq([1, 2, 2, 3, 5])
pmf.mean(), pmf.var()

In [151]:
# Solution goes here

In [152]:
# Solution goes here

**Exercise:** Something like the class size paradox appears if you survey children and ask how many children are in their family. Families with many children are more likely to appear in your sample, and families with no children have no chance to be in the sample.

Use the NSFG respondent variable `numkdhh` to construct the actual distribution for the number of children under 18 in the respondents' households.

Now compute the biased distribution we would see if we surveyed the children and asked them how many children under 18 (including themselves) are in their household.

Plot the actual and biased distributions, and compute their means.

In [153]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/2002FemResp.dct")
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/2002FemResp.dat.gz")

In [154]:
resp = nsfg.read_fem_resp()

In [156]:
# Solution goes here

In [157]:
# Solution goes here

In [159]:
# Solution goes here

In [160]:
# Solution goes here

In [161]:
# Solution goes here

In [162]:
# Solution goes here

**Exercise:** I started this book with the question, "Are first babies more likely to be late?" To address it, I computed the difference in means between groups of babies, but I ignored the possibility that there might be a difference between first babies and others for the same woman.

To address this version of the question, select respondents who have at least two live births and compute pairwise differences. Does this formulation of the question yield a different result?

In [163]:
live, firsts, others = nsfg.make_frames()

In [164]:
preg_map = nsfg.make_preg_map(live)

In [165]:
# Solution goes here

In [166]:
# Solution goes here

In [167]:
# Solution goes here

**Exercise:** In most foot races, everyone starts at the same time. If you are a fast runner, you usually pass a lot of people at the beginning of the race, but after a few miles everyone around you is going at the same speed.
When I ran a long-distance (209 miles) relay race for the first time, I noticed an odd phenomenon: when I overtook another runner, I was usually much faster, and when another runner overtook me, he was usually much faster.

At first I thought that the distribution of speeds might be bimodal; that is, there were many slow runners and many fast runners, but few at my speed.

Then I realized that I was the victim of a bias similar to the effect of class size. The race was unusual in two ways: it used a staggered start, so teams started at different times; also, many teams included runners at different levels of ability.

As a result, runners were spread out along the course with little relationship between speed and location. When I joined the race, the runners near me were (pretty much) a random sample of the runners in the race.

So where does the bias come from? During my time on the course, the chance of overtaking a runner, or being overtaken, is proportional to the difference in our speeds. I am more likely to catch a slow runner, and more likely to be caught by a fast runner. But runners at the same speed are unlikely to see each other.

Write a function called `ObservedPmf` that takes a `Pmf` representing the actual distribution of runners' speeds, and the speed of a running observer, and returns a new `Pmf` representing the distribution of runners' speeds as seen by the observer.

To test your function, you can use `relay.py`, which reads the results from the James Joyce Ramble 10K in Dedham, Massachusetts and converts the pace of each runner to mph.

Compute the distribution of speeds you would observe if you ran a relay race at 7 mph with this group of runners.

In [168]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/relay.py")
download(
    "https://github.com/AllenDowney/ThinkStats/raw/v3/data/Apr25_27thAn_set1.shtml"
)

In [169]:
import relay

results = relay.read_results()
speeds = relay.get_speeds(results)
speeds = relay.bin_data(speeds, 3, 12, 100)

In [172]:
# Solution goes here

In [177]:
# Solution goes here

In [178]:
# Solution goes here